In [1]:
import requests
import json
import hmac
import hashlib
import base64
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import Workbook
import os
from datetime import datetime

# 네이버 검색량 API 관련 설정
API_KEY = "0100000000f7219646b1189747046631aa4d35e62954247e750a216e37f7072b5174cbe5c2"
SECRET_KEY = "AQAAAAD3IZZGsRiXRwRmMapNNeYpncLjvqyOrRpn2cZDTV49Ug=="
CUSTOMER_ID = "373124"

def generate_signature(timestamp, method, uri, secret_key):
    try:
        message = f"{timestamp}.{method}.{uri}"
        signature = hmac.new(secret_key.encode(), message.encode(), hashlib.sha256).digest()
        signature_b64 = base64.b64encode(signature).decode()
        return signature_b64
    except Exception as e:
        print(f"Error generating signature: {e}")
        return None

def get_naver_search_volume(api_key, secret_key, customer_id, keyword):
    try:
        timestamp = str(int(time.time() * 1000))
        method = "GET"
        uri = "/keywordstool"
        signature = generate_signature(timestamp, method, uri, secret_key)
        
        if not signature:
            return None

        url = "https://api.naver.com" + uri
        headers = {
            "X-API-KEY": api_key,
            "X-API-SECRET": secret_key,
            "X-CUSTOMER": customer_id,
            "X-Timestamp": timestamp,
            "X-Signature": signature,
            "Content-Type": "application/json",
            "Accept": "*/*"
        }
        params = {
            "hintKeywords": keyword,
            "showDetail": "1"
        }
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if data and "keywordList" in data and len(data["keywordList"]) > 0:
                item = data["keywordList"][0]
                monthlyPcQcCnt = item.get("monthlyPcQcCnt", 0)
                monthlyMobileQcCnt = item.get("monthlyMobileQcCnt", 0)

                if isinstance(monthlyPcQcCnt, str) and '<' in monthlyPcQcCnt:
                    monthlyPcQcCnt = 5  # '< 10'인 경우, 5로 설정 (임의 값)
                else:
                    monthlyPcQcCnt = int(monthlyPcQcCnt)

                if isinstance(monthlyMobileQcCnt, str) and '<' in monthlyMobileQcCnt:
                    monthlyMobileQcCnt = 5  # '< 10'인 경우, 5로 설정 (임의 값)
                else:
                    monthlyMobileQcCnt = int(monthlyMobileQcCnt)

                total_search_volume = monthlyPcQcCnt + monthlyMobileQcCnt
                return total_search_volume
        else:
            print(f"Error: {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"Error fetching search volume: {e}")
        return None

# 현재 날짜를 yyyymmdd 형식으로 얻기
current_date = datetime.now().strftime("%Y%m%d")
search_date = datetime.now().strftime("%Y-%m-%d")  # 검색 날짜를 yyyy-mm-dd 형식으로 얻기

# 사용자로부터 저장할 Excel 파일 기본 이름 입력받기
base_file_name = input("저장할 Excel 파일의 기본 이름을 입력하세요 (확장자 제외): ")

# 입력받은 파일 이름에 오늘 날짜 추가
excel_file_name = f"{base_file_name}_{current_date}.xlsx"

# 현재 작업 중인 폴더의 경로를 얻기
current_working_directory = os.getcwd()
file_path = os.path.join(current_working_directory, excel_file_name)  # 파일 경로 결합

# WebDriver 설정(Chrome 사용)
driver = webdriver.Chrome()

# Excel 설정
wb = Workbook()
ws = wb.create_sheet('검색어순위')
wb.remove(wb['Sheet'])  # 기본 생성된 시트 제거
ws.append(['순위', '인기검색어', '검색 날짜', '월간총검색량'])

# 네이버 데이터랩 쇼핑 인사이트 페이지로 이동
driver.get("https://datalab.naver.com/shoppingInsight/sCategory.naver")

# 필요한 요소가 로드될 때까지 대기하는 WebDriverWait 인스턴스 생성
wait = WebDriverWait(driver, 20)

# 사용자에게 브라우저에서 옵션 선택을 요청
print("브라우저에서 원하는 옵션을 선택한 후, 이 콘솔에서 Enter 키를 눌러주세요...")
input("옵션 선택을 완료한 후 Enter 키를 눌러주세요...")

# 조회 버튼 클릭
try:
    search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/a')))
    search_button.click()
except Exception as e:
    print(f"Error occurred while clicking the search button: {e}")

# 사용자로부터 페이지 수 입력받기
while True:
    try:
        page_count = int(input("크롤링할 페이지 수를 입력하세요 (1-25): "))
        if 1 <= page_count <= 25:
            break
        else:
            print("페이지 수는 1에서 25 사이의 정수여야 합니다.")
    except ValueError:
        print("유효한 정수를 입력하세요.")

# 결과 스크래핑 및 Excel 파일 저장
for i in range(page_count):  # 사용자로부터 입력받은 페이지 수만큼 반복
    # 페이지 로딩 대기
    time.sleep(1)
    
    for j in range(1, 21):  # 한 페이지당 최대 20개 아이템
        try:
            path = f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[1]/ul/li[{j}]/a'
            result = driver.find_element(By.XPATH, path).text
            result_data = result.split('\n')
            result_data[0] = int(result_data[0])  # 순위를 숫자 형식으로 변환
            keyword = result_data[1]
            search_volume = get_naver_search_volume(API_KEY, SECRET_KEY, CUSTOMER_ID, keyword)
            result_data.append(search_date)  # 검색 날짜 추가
            result_data.append(search_volume if search_volume is not None else "N/A")  # 검색량 추가
            print(result_data)
            ws.append(result_data)
        except Exception as e:
            print(f"Error occurred while scraping data: {e}")
    
    # 다음 페이지로 넘어가는 버튼 클릭 로직 (실제 페이지 구조에 따라 달라질 수 있음)
    try:
        next_button = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/a[2]')
        next_button.click()
        time.sleep(1)  # 페이지 로딩 대기
    except Exception as e:
        print(f"Error occurred while clicking next button: {e}")
        break

# 파일 저장
try:
    wb.save(file_path)
    wb.close()
    print(f"'{excel_file_name}' 파일이 현재 폴더에 저장되었습니다.")
except Exception as e:
    print(f"Error occurred while saving the file: {e}")

# 작업 완료 후, 드라이버 종료
driver.quit()

# 저장된 파일 열기
try:
    os.startfile(file_path)
except Exception as e:
    print(f"Error occurred while opening the file: {e}")


저장할 Excel 파일의 기본 이름을 입력하세요 (확장자 제외): 뜨개실검색량
브라우저에서 원하는 옵션을 선택한 후, 이 콘솔에서 Enter 키를 눌러주세요...
옵션 선택을 완료한 후 Enter 키를 눌러주세요...
크롤링할 페이지 수를 입력하세요 (1-25): 10
[1, '메탈실', '2024-06-19', 7800]
[2, '뜨개실', '2024-06-19', 15360]
[3, '얀메이크', '2024-06-19', 17940]
[4, '라라뜨개', '2024-06-19', 11830]
[5, '니트컨테이너', '2024-06-19', 9490]
[6, '라피아실', '2024-06-19', 8870]
[7, '코바늘가방', '2024-06-19', 6540]
[8, '뜨개질', '2024-06-19', 18440]
[9, '수세미실', '2024-06-19', 6260]
[10, '코바늘', '2024-06-19', 13020]
[11, '메탈릭실', '2024-06-19', 3930]
[12, '청송뜨개실', '2024-06-19', 28220]
[13, '루피실', '2024-06-19', 5760]
[14, '코바늘diy', '2024-06-19', 3250]
[15, '뜨개사계절', '2024-06-19', 12750]
[16, '바늘이야기', '2024-06-19', 79500]
[17, '뜨개질가방', '2024-06-19', 4510]
[18, '네트백뜨기', '2024-06-19', 3700]
[19, '코바늘가방뜨기', '2024-06-19', 6960]
[20, '실버실', '2024-06-19', 2930]
[21, '야닝야닝', '2024-06-19', 4530]
[22, '갤럭시메탈릭코드', '2024-06-19', 2290]
[23, '얀메이크착하면', '2024-06-19', 4570]
[24, '메탈릭얀', '2024-06-19', 2880]
[25, '앵콜스뜨개실', '2024-06-19', 18420]
[26, '뜨개